In [1]:
import pandas as pd

In [2]:
path ="data.xlsx"
#interventions_df = pd.read_csv("interventions.tsv",sep='\t',index_col=0 )
#outputs_df = pd.read_csv("outputs.tsv",sep='\t',index_col=0)
#individuals_df = pd.read_csv("individuals.tsv",sep='\t',index_col=[0,1,2],header=[0,1] )
#groups_df = pd.read_csv("groups.tsv",sep='\t',index_col=[0,1,2],header=[0,1] )
#subjects_df = pd.read_csv("all_subjects.tsv",sep='\t',index_col=[0,1,2,3],header=[0,1] )
interventions_df = pd.read_excel(path,sheet_name="interventions",index_col=[0,1,3])
outputs_df = pd.read_excel(path,sheet_name="outputs",index_col=[0,1])
timecourses_df = pd.read_excel(path,sheet_name="timecourses",index_col=[0,1])
individuals_df = pd.read_excel(path,sheet_name="individuals",index_col=[0,1,2],header=[0,1])
groups_df = pd.read_excel(path,sheet_name="groups",index_col=[0,1,2],header=[0,1])

In [3]:
def convert_unit(df, unit_in, unit_out, factor=1.0, 
                 unit_field="unit", data_fields=['mean','median','value', 'sd', 'se', 'min', 'max'], inplace=True, subset=False):
    """ Unit conversion in given data frame. """
    if not inplace:
        df = df.copy()
    if subset:
        is_weightidx =  df[subset].notnull()
        df =  df[is_weightidx]     
    if isinstance(factor, pd.Series):
        factor = factor[is_weightidx]
        
        
    idx = (df[unit_field] == unit_in)
    

    for key in data_fields:
        df.loc[idx, key] = df.loc[idx, key]*factor
    df.loc[idx, unit_field] = unit_out
    
    if subset:
        return df[idx]
    
    return df
    

# thalf & tmax [min] -> [h]
convert_unit(outputs_df, unit_in='min', unit_out='h', factor=1/60.0)
# cmax [mg/l] -> [µg/ml]
convert_unit(outputs_df, unit_in='mg/l', unit_out='µg/ml', factor=1.0)

# clearance ['ml/h/kg'] -> ['ml/min/kg']
convert_unit(outputs_df, unit_in='ml/h/kg', unit_out='ml/min/kg', factor=1/60.0)
convert_unit(outputs_df, unit_in='l/h/kg', unit_out='ml/min/kg', factor=1000/60.0)
convert_unit(outputs_df, unit_in='l/h', unit_out='ml/min', factor=1000.0/60)

# auc ['mg/l*min'] -> ['mg/l*h']
convert_unit(outputs_df, unit_in='mg/l*min', unit_out='mg/l*h', factor=1/60.0)

# auc ['mg/l*h'] -> ['µg/ml*h']
#convert_unit(outputs_df, unit_in='mg/l*h', unit_out='µg/ml*h', factor=1.0)
# overview of units for given pktype
pktypes = sorted(outputs_df.pktype.unique())
print('-'*80)
print('PKPD data ({})'.format(len(outputs_df)))
print('-'*80)
for pktype in pktypes:
    data = outputs_df[outputs_df.pktype==pktype]
    print('{:<6} {:<20}{}'.format(len(data), pktype, sorted(data.unit.unique())))
print('-'*80)

--------------------------------------------------------------------------------
PKPD data (2088)
--------------------------------------------------------------------------------
50     auc_end             ['mg*h/l', 'ng*h/ml', 'µg*h/ml', 'µg/ml*h']
169    auc_inf             ['mg*h/l', 'ng*h/ml', 'nmol*h/l', 'µg*h/ml', 'µg*min/ml', 'µg/ml*h', 'µmol*h/l', 'µmol/l*h']
500    clearance           ['ml/min', 'ml/min/1.73m^2', 'ml/min/kg']
22     clearance_renal     ['ml/min']
5      clearance_unbound   ['ml/min/kg']
106    cmax                ['ng/ml', 'nmol/l', 'µg/ml', 'µmol/l']
80     concentration       ['µg/ml']
1      fraction_absorbed   ['-']
5      kabs                ['1/h']
110    kel                 ['1/h']
5      plasma_binding      ['-']
279    ratio               ['-']
104    recovery            ['%', '-']
359    thalf               ['h']
93     tmax                ['h']
200    vd                  ['l', 'l/kg']
-----------------------------------------------------------------

In [4]:
print(len(outputs_df), 'pkpd outputs')
# merge with dose information
pk = pd.merge(left=outputs_df, right=interventions_df,  how='inner', suffixes=('','_intervention'),left_on='interventions', right_on="pk")
print(len(pk), 'merge interventions')

# merge with subject information
pk_i = pd.merge(pk,individuals_df.reset_index(),  how='inner', suffixes=('','individual'),left_on='individual_pk', right_on="subject_pk")
pk_g = pd.merge(pk,groups_df.reset_index(),  how='inner', suffixes=('','group'),left_on='group_pk', right_on="subject_pk")
print(len(pk_i), 'merge individuals')
print(len(pk_g), 'merge groups')
print(len(pk_i)+len(pk_g), 'both')

2088 pkpd outputs
2088 merge interventions
1438 merge individuals
650 merge groups
2088 both


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


All males donot take oral conraceptives

In [5]:
individuals_df.loc[(individuals_df["sex"]["choice"] == "M"),("oral contraceptives","choice")] = "N"
groups_df.loc[(groups_df["sex"]["choice"] == "M"),("oral contraceptives","choice")] = "N"

## Merging datasets

In [6]:
print(len(outputs_df), 'pkpd outputs')
# merge with dose information
pk = pd.merge(left=outputs_df, right=interventions_df,  how='inner', suffixes=('','_intervention'),left_on='interventions', right_on="pk")
print(len(pk), 'merged outputs and interventions')

# merge with subject information
pk_i = pd.merge(pk,individuals_df.reset_index(),  how='inner', suffixes=('','individual'),left_on='individual_pk', right_on="subject_pk")
pk_g = pd.merge(pk,groups_df.reset_index(),  how='inner', suffixes=('','group'),left_on='group_pk', right_on="subject_pk")
print(len(pk_i), 'merged outputs and individuals')
print(len(pk_g), 'merged outputs and groups')
print(len(pk_i)+len(pk_g), 'both')

2088 pkpd outputs
2088 merged outputs and interventions
1438 merged outputs and individuals
650 merged outputs and groups
2088 both


## Adding:  Recalculated data (inteventions relative to weight and absolute, outputs relative to weight and absolute)

for individuals

In [7]:
pk_i_realtive = convert_unit(pk_i,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/pk_i[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=("weight","value"))

print("Number of recalcualted measurements with intervention unit  mg and bodyweight:" ,len(pk_i_realtive))

pk_i_absolute = convert_unit(pk_i,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=pk_i[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=("weight","value"))

print("Number of recalcualted measurements with intervention unit  mg/kg and bodyweight:" ,len(pk_i_absolute))

pk_i = pd.concat([pk_i,pk_i_realtive,pk_i_absolute], ignore_index=True)

print("added to remaining:" ,len(pk_i))

pk_i_realtive = convert_unit(pk_i,
                             unit_in="ml/min",
                             unit_out="ml/min/kg", 
                             factor=1.0/pk_i[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=("weight","value"))

print("Number of recalcualted measurements with output unit ml/min and bodyweight:" ,len(pk_i_realtive))

pk_i_absolute = convert_unit(pk_i,
                             unit_in="ml/min/kg",
                             unit_out="ml/min", 
                             factor=pk_i[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'],
                             subset=("weight","value"))

print("Number of recalcualted measurements with output unit ml/min/kg and bodyweight" ,len(pk_i_absolute))

pk_i = pd.concat([pk_i,pk_i_realtive,pk_i_absolute], ignore_index=True)
print("added to remaining:" ,len(pk_i))

Number of recalcualted measurements with intervention unit  mg and bodyweight: 654
Number of recalcualted measurements with intervention unit  mg/kg and bodyweight: 97
added to remaining: 2189


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Number of recalcualted measurements with output unit ml/min and bodyweight: 60
Number of recalcualted measurements with output unit ml/min/kg and bodyweight 350
added to remaining: 2599


for groups

In [8]:
pk_g_realtive = convert_unit(pk_g,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/pk_g[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=("weight","mean"))

print("Number of recalcualted measurements with intervention unit  mg and bodyweight:" ,len(pk_g_realtive))

pk_g_absolute = convert_unit(pk_g,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=pk_g[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=("weight","mean"))
print("Number of recalcualted measurements with intervention unit  mg/kg and bodyweight:" ,len(pk_g_absolute))

pk_g = pd.concat([pk_g,pk_g_realtive,pk_g_absolute], ignore_index=True)
print("added to remaining:" ,len(pk_g))

pk_g_realtive = convert_unit(pk_g,
                             unit_in="ml/min",
                             unit_out="ml/min/kg", 
                             factor=1.0/pk_g[("weight","mean")],
                             unit_field="unit", 
                             subset=("weight","mean"))

print("Number of recalcualted measurements with output unit ml/min and bodyweight:" ,len(pk_g_realtive))

pk_g_absolute = convert_unit(pk_g,
                             unit_in="ml/min/kg",
                             unit_out="ml/min", 
                             factor=pk_g[("weight","mean")],
                             unit_field="unit", 
                             subset=("weight","mean"))
print("Number of recalcualted measurements with output unit ml/min/kg and bodyweight:" ,len(pk_g_absolute))

pk_g = pd.concat([pk_g,pk_g_realtive,pk_g_absolute], ignore_index=True)
print("added to remaining:" ,len(pk_g))

Number of recalcualted measurements with intervention unit  mg and bodyweight: 186
Number of recalcualted measurements with intervention unit  mg/kg and bodyweight: 213
added to remaining: 1049
Number of recalcualted measurements with output unit ml/min and bodyweight: 40
Number of recalcualted measurements with output unit ml/min/kg and bodyweight: 68
added to remaining: 1157


In [9]:
from openpyxl import load_workbook

book = load_workbook('data.xlsx')
writer = pd.ExcelWriter('data.xlsx')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

pk_i.to_excel(writer,"complete_for_individuals")

pk_g.to_excel(writer,"complete_for_groups")

writer.save()
writer.close()

In [10]:
pk_i.to_csv("complete_for_individuals.tsv", sep="\t")
pk_g.to_csv("complete_for_groups.tsv", sep="\t")
